In [136]:
import MetaTrader5 as mt5
from datetime import datetime
import time
import pytz
import pandas as pd

In [137]:
def Mt5_dados(tickers,inicio,fim):

    # conecte-se ao MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()
    else:
        # consulta o estado e os parâmetros de conexão
        print(mt5.terminal_info())
    
        #quebra as strings dates(inicio,fim) recebidas e transformando em integer
        inicio_= inicio.split('-')
        inicio = [int(element) for element in inicio_]
        fim_= fim.split('-')
        fim = [int(element) for element in fim_]

        timezone = pytz.timezone("Etc/UTC")

        # obtendo o par de acoes com base nas datas enviadas nas strings, definindo o timezone para nao interfir com o tisp do mt5
        stock1 = mt5.copy_rates_range(tickers[0], mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),datetime(fim[0],fim[1],fim[2]))
        stock2 = mt5.copy_rates_range(tickers[1], mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),datetime(fim[0],fim[1],fim[2]))
        # concluímos a conexão ao MetaTrader 5
        mt5.shutdown()

        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})

        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)

    return stock_df.set_index('Date')

In [138]:
tickers = ['ITSA4', 'GOLL4'];
par = Mt5_dados(tickers,'2019-1-2','2020-7-27')

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2560, maxbars=100000, codepage=1252, ping_last=11139, community_balance=0.0, retransmission=0.20934783598052153, company='Rico (XP Investimentos CCTVM)', name='Rico - MetaTrader 5', language='Portuguese (Brazil)', path='C:\\Program Files\\Rico - MetaTrader 5', ...)


In [139]:
par

,ITSA4,GOLL4
Date,,
2019-01-02,11.18,25.54
2019-01-03,11.28,25.72
2019-01-04,11.24,25.09
2019-01-07,11.23,23.99
2019-01-08,11.38,23.67
...,...,...
2020-07-21,10.52,20.66
2020-07-22,10.52,20.00
2020-07-23,10.37,19.10
